# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
!pip install langchain

In [2]:
!pip install langchain-openai

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [5]:
#!pip install pandas

In [10]:
import pandas as pd
df = pd.read_csv('Data.csv')

In [11]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\n,I loved this product. But they only seem to l...


## LLMChain

In [12]:
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain

In [13]:
#Replace None by your own value and justify - balance between creativity and determinism
llm = ChatOpenAI(temperature=0.5) 


In [14]:
prompt = ChatPromptTemplate.from_template(  "Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

In [15]:
# old code 
#chain = LLMChain(llm=llm, prompt=prompt)
#correct code
chain = prompt | llm


In [16]:
product = "shower hat"
chain.invoke(product)

AIMessage(content='Introducing the Shower Hat - the ultimate solution for those bad hair days in the shower! Say goodbye to frizzy hair and hello to smooth locks with this stylish and waterproof hat. Its adjustable strap ensures a snug fit for all head sizes, while the fun design adds a touch of flair to your daily routine. Shower in style and keep your hair looking fabulous - no umbrella required!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 77, 'prompt_tokens': 27, 'total_tokens': 104, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-97c44f64-9492-4c3b-8be2-a419914c5312-0', usage_metadata={'input_tokens': 27, 'output_tokens': 77, 'total_tokens': 104, 'input_token_det

## SimpleSequentialChain

In [17]:
from langchain.chains import SimpleSequentialChain

In [18]:
llm = ChatOpenAI(temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
"Write a funny product description for {product}. "
    "Highlight its key features and benefits. Be short and concise"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

/var/folders/4r/064fnvjs2j9g2l9n63gy6k0w0000gn/T/ipykernel_29423/1498939052.py:10: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain_one = LLMChain(llm=llm, prompt=first_prompt)


In [19]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the review {review}, rate the quality of this product from zero to ten."
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [20]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [21]:
overall_simple_chain.invoke(product)



> Entering new SimpleSequentialChain chain...
Introducing the Shower Hat: the ultimate accessory for keeping your hair dry and fabulous in the shower!

Key Features:
- Waterproof and durable design
- Elastic band for a snug and comfortable fit
- Cute and stylish design to make shower time fun

Benefits:
- Say goodbye to frizzy, wet hair after showering
- Save time and energy by avoiding the hassle of blow-drying your hair
- Perfect for protecting your hairstyle while still getting clean

Get your Shower Hat today and shower in style!
I would rate this product a 9 out of 10. The key features such as the waterproof and durable design, elastic band for a comfortable fit, and cute design are all great selling points. The benefits of avoiding frizzy hair, saving time on blow-drying, and protecting hairstyles are also very appealing. Overall, the Shower Hat seems like a practical and stylish solution for keeping hair dry in the shower.

> Finished chain.


{'input': 'shower hat',
 'output': 'I would rate this product a 9 out of 10. The key features such as the waterproof and durable design, elastic band for a comfortable fit, and cute design are all great selling points. The benefits of avoiding frizzy hair, saving time on blow-drying, and protecting hairstyles are also very appealing. Overall, the Shower Hat seems like a practical and stylish solution for keeping hair dry in the shower.'}

**Repeat the above twice for different products**

## SequentialChain

In [22]:
from langchain.chains import SequentialChain

In [23]:
llm = ChatOpenAI(temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
  "Translate this {review} into french. "
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key='fr_review' #Give a name to your output
                    )


In [24]:
second_prompt = ChatPromptTemplate.from_template(
    'Based on {fr_review}, summarize this review in less than 10 words in french.'
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                     output_key='fr_short_review' #give a name to this output
                    )


In [25]:
# prompt template 3: translate to english or other language
third_prompt = ChatPromptTemplate.from_template(
    "Translate {fr_short_review} into english"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                       output_key='en_short_review'
                      )


In [26]:

# prompt template 4: follow up message that take as inputs the two previous prompts' variables
fourth_prompt = ChatPromptTemplate.from_template(
"Rate the quality of the translation of {fr_short_review} into {en_short_review} from zero to ten."
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key='rating'
                     )


In [27]:
# overall_chain: input= Review 
# and output= English_Review,summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["review"],
    output_variables=['fr_short_review','en_short_review', 'rating'],
    verbose=True
)

In [28]:
review = df.Review[5]
overall_chain.invoke(review)



> Entering new SequentialChain chain...

> Finished chain.


{'review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\nVieux lot ou contrefaçon !?",
 'fr_short_review': 'Goût médiocre, mousse instable, probablement contrefaçon.',
 'en_short_review': 'Poor taste, unstable foam, probably counterfeit.',
 'rating': 'I would rate the quality of the translation as a 9 out of 10. It accurately conveys the meaning of the original French sentence while maintaining a similar tone and structure.'}

**Repeat the above twice for different products or reviews**

## Router Chain

In [29]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

In [30]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    }
]

prompt_templates = {
    info["name"]: ChatPromptTemplate.from_template(info["prompt_template"])
    for info in prompt_infos
}


In [31]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.prompts import PromptTemplate

In [32]:
llm = ChatOpenAI(temperature=0)

In [33]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [34]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [35]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [36]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)

router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

#router_chain = router_prompt | llm | RunnableLambda(lambda x: x.content) 
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, 
                         verbose=True
                        )

/var/folders/4r/064fnvjs2j9g2l9n63gy6k0w0000gn/T/ipykernel_29423/2426684390.py:16: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [37]:
router_chain.invoke("Explain what principle component analysis is using linear algebra")

{'input': 'Explain what principle component analysis is using linear algebra',
 'destination': 'math',
 'next_inputs': {'input': 'Explain what principle component analysis is using linear algebra'}}

In [38]:
response = chain.invoke("Explain what principle component analysis is using linear algebra")




> Entering new MultiPromptChain chain...
computer science: {'input': 'Explain what principle component analysis is using linear algebra'}
> Finished chain.


In [39]:
response

{'input': 'Explain what principle component analysis is using linear algebra',
 'text': 'Principal Component Analysis (PCA) is a technique used in data analysis and machine learning to reduce the dimensionality of a dataset while preserving as much variance as possible. \n\nIn linear algebra terms, PCA involves finding the eigenvectors and eigenvalues of the covariance matrix of the dataset. The eigenvectors represent the directions of maximum variance in the data, while the eigenvalues represent the amount of variance along each eigenvector.\n\nTo perform PCA, we first center the data by subtracting the mean of each feature from the data points. Then, we compute the covariance matrix of the centered data. Next, we calculate the eigenvectors and eigenvalues of the covariance matrix. \n\nThe eigenvectors form a new basis for the data, where the first eigenvector corresponds to the direction of maximum variance, the second eigenvector corresponds to the direction of second maximum varian

In [40]:
chain.invoke("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


{'input': 'What is black body radiation?',
 'text': "Black body radiation is the electromagnetic radiation emitted by a perfect absorber of radiation, known as a black body. A black body absorbs all incoming radiation and emits radiation across the entire electromagnetic spectrum. The spectrum of black body radiation is continuous and follows a specific distribution known as Planck's law. This phenomenon is important in understanding the behavior of objects at different temperatures and is a key concept in the field of thermal physics."}

In [41]:
chain.invoke("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


{'input': 'what is 2 + 2', 'text': 'The answer to 2 + 2 is 4.'}

In [42]:
chain.run("Why does mexico speak spanish?")

/var/folders/4r/064fnvjs2j9g2l9n63gy6k0w0000gn/T/ipykernel_29423/3386253788.py:1: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("Why does mexico speak spanish?")




> Entering new MultiPromptChain chain...
History: {'input': 'Why does Mexico speak Spanish?'}
> Finished chain.


'Mexico speaks Spanish because of its history of colonization by the Spanish Empire. In the early 16th century, Spanish conquistadors led by Hernan Cortes invaded and conquered the Aztec Empire, establishing New Spain as a colony of Spain. The Spanish language was imposed on the indigenous populations as part of the process of colonization and assimilation.\n\nOver time, Spanish became the dominant language in Mexico, supplanting the indigenous languages spoken by the native populations. Today, Spanish is the official language of Mexico and is spoken by the majority of the population. The legacy of Spanish colonization is also evident in Mexican culture, religion, and society.'

**Repeat the above at least once for different inputs and chains executions - Be creative!**